In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime


In [52]:
#%% 0. Housekeeping 

# =============================================================================
# 0.1 Import packages
# =============================================================================

import pandas as pd
import datetime as dt
from datetime import datetime
   
# =============================================================================
# 0.2 Import data
# =============================================================================

train = pd.read_csv('data/train.csv', low_memory=False)
store = pd.read_csv('data/store.csv', low_memory=False)

#%% 1. Merging store to train data

# =============================================================================
# 1.1 Since data has to have the same size as 'train', a container is created
# =============================================================================

# Creating dataframe
expanded_store = train

# =============================================================================
# 1.2 Merging variables which do not have to be changed
# =============================================================================

# Variables which can be merged right away
easy = store.loc[:,['Store','StoreType','Assortment','CompetitionDistance']]

# Variables which can be simply merged
expanded_store = pd.merge(expanded_store, easy, on=['Store'])

#%% 2. Creating a dummy variable since for competition is open for every store

# =============================================================================
# 2.1 Checking whether there is actually competition
# =============================================================================

# Dataset of the variables which have to be transformed
competition = store.loc[:,['Store',
                           'CompetitionOpenSinceMonth','CompetitionOpenSinceYear']]

# For easier looping 
competition.set_index('Store', inplace=True)

# Finding stores which have competition openend
open_comp = competition.any(axis=1)

# Stores with competition
open_stores = competition.loc[open_comp].index.unique()

# =============================================================================
# 2.2 Generating dummy for the time competition is existing
# =============================================================================

# Generating Date when opened
for store_id in open_stores: 
    year = competition.loc[store_id,'CompetitionOpenSinceYear'].astype(int)
    month = competition.loc[store_id,'CompetitionOpenSinceMonth'].astype(int)
    competition.loc[store_id,'CompetitionDate'] = datetime(year,month,1)

# Creating a dummy variable for whether competition openend for each store
expanded_store.loc[:,'CompetitionOpened'] = 0

for store_id in open_stores: 
    
    # Getting the date when competition openend
    date = competition.loc[store_id,'CompetitionDate']
    date_str= date.strftime("%Y-%m-%d")
    
    store_number = expanded_store.loc[:,'Store'] == store_id
    
    maximum_date = expanded_store.loc[store_number,'Date'].max()
    
    # Whether it falls in time frame
    competition_existing = (expanded_store.loc[:,'Date'].between(date_str,maximum_date)) &  (expanded_store.loc[:,'Store'] == store_id)
    
    # Indicating whether competition is around
    expanded_store.loc[competition_existing, 'CompetitionOpened'] = 1


#%% 3. Creating a dummy variable for Promo
 
# =============================================================================
# 3.1 Since data has to have the same size as 'train', a container is created
# =============================================================================

# Create dataframe
promo2 = store.loc[:,['Store',
                     'Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval']]

# =============================================================================
# 3.2 Create dummy for whether a promo2 is running 
# =============================================================================

# Getting date from which promo started
length = promo2.shape[0]
for i in range(length): 
    if promo2.loc[i,'Promo2']:
        week = promo2.loc[i,'Promo2SinceWeek'].astype(int)
        year = promo2.loc[i,'Promo2SinceYear'].astype(int)
        promo2.loc[i,'promo2start'] = dt.datetime.strptime(f'{year}-W{int(week )- 1}-1', "%Y-W%W-%w").date()

# Merge it with the train file
expanded_promo = pd.merge(expanded_store, promo2, on=['Store'])


# Empty container with no promo indicator
expanded_promo.loc[:,'Promo2GoingOn'] = 0

# Month indication
expanded_promo.loc[:,'Date_str'] = pd.to_datetime(expanded_promo.loc[:,'Date'],).dt.strftime('%Y-%b-%d')
expanded_promo.loc[:,'Date'] = pd.to_datetime(expanded_promo.loc[:,'Date'],)
expanded_promo.loc[:,'month'] = expanded_promo.loc[:,'Date_str'].str[5:8]


months = expanded_promo.loc[:,'month'].unique()

for month in months:
    month_boolean = expanded_promo.loc[:,'PromoInterval'].str.contains(month, na=False)
    expanded_promo.loc[month_boolean,'Promo2GoingOn'] = 1

In [53]:
expanded=expanded_promo
expanded.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpened', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'promo2start', 'Promo2GoingOn',
       'Date_str', 'month'],
      dtype='object')

In [54]:
# Convert Dates to Date time
expanded['Date']=pd.to_datetime(expanded['Date'])
expanded.set_index('Date')

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpened,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,promo2start,Promo2GoingOn,Date_str,month
Date,,,,,,,,,,,,,,,,,,,,
2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0,d,c,5350.0,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012-05-21,1,2013-Jan-01,Jan
2013-01-02,1115.0,3.0,3697.0,305.0,1.0,0.0,0,1.0,d,c,5350.0,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012-05-21,1,2013-Jan-02,Jan
2013-01-03,1115.0,4.0,4297.0,300.0,1.0,0.0,0,1.0,d,c,5350.0,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012-05-21,1,2013-Jan-03,Jan
2013-01-04,1115.0,5.0,4540.0,326.0,1.0,0.0,0,1.0,d,c,5350.0,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012-05-21,1,2013-Jan-04,Jan
2013-01-05,1115.0,6.0,4771.0,339.0,1.0,0.0,0,1.0,d,c,5350.0,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012-05-21,1,2013-Jan-05,Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-06-26,127.0,4.0,3335.0,480.0,1.0,0.0,0,0.0,d,a,1350.0,1,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2010-03-22,1,2014-Jun-26,Jun
2014-06-27,127.0,5.0,4724.0,591.0,1.0,0.0,0,0.0,d,a,1350.0,1,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2010-03-22,1,2014-Jun-27,Jun
2014-06-28,127.0,6.0,5398.0,716.0,1.0,0.0,0,0.0,d,a,1350.0,1,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2010-03-22,1,2014-Jun-28,Jun


In [55]:
#add in variables for day of month etc
expanded['dayofweek'] = expanded['Date'].dt.dayofweek
expanded['quarter'] = expanded['Date'].dt.quarter
expanded['month'] = expanded['Date'].dt.month
expanded['year'] = expanded['Date'].dt.year
expanded['dayofyear'] = expanded['Date'].dt.dayofyear
expanded['dayofmonth'] = expanded['Date'].dt.day
expanded['weekofyear'] = expanded['Date'].dt.weekofyear

In [56]:
#create dummy variables for day of week etc and categorical variables
expanded= pd.get_dummies(expanded)
expanded= pd.get_dummies(expanded,columns=['dayofweek','dayofmonth','quarter','month'])
expanded.set_index('Date')


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpened,Promo2,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-01-02,1115.0,3.0,3697.0,305.0,1.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-01-03,1115.0,4.0,4297.0,300.0,1.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-01-04,1115.0,5.0,4540.0,326.0,1.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-01-05,1115.0,6.0,4771.0,339.0,1.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-06-26,127.0,4.0,3335.0,480.0,1.0,0.0,0.0,1350.0,1,1,...,0,0,0,1,0,0,0,0,0,0
2014-06-27,127.0,5.0,4724.0,591.0,1.0,0.0,0.0,1350.0,1,1,...,0,0,0,1,0,0,0,0,0,0
2014-06-28,127.0,6.0,5398.0,716.0,1.0,0.0,0.0,1350.0,1,1,...,0,0,0,1,0,0,0,0,0,0


In [57]:
#Re-add original day of month variable etc.
expanded['dayofweek'] = expanded['Date'].dt.dayofweek
expanded['quarter'] = expanded['Date'].dt.quarter
expanded['month'] = expanded['Date'].dt.month
expanded['year'] = expanded['Date'].dt.year
expanded['dayofyear'] = expanded['Date'].dt.dayofyear
expanded['dayofmonth'] = expanded['Date'].dt.day
expanded['weekofyear'] = expanded['Date'].dt.weekofyear

expanded.set_index('Date')


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpened,Promo2,...,month_7,month_8,month_9,month_10,month_11,month_12,dayofweek,quarter,month,dayofmonth
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,1,1,1,1
2013-01-02,1115.0,3.0,3697.0,305.0,1.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,2,1,1,2
2013-01-03,1115.0,4.0,4297.0,300.0,1.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,3,1,1,3
2013-01-04,1115.0,5.0,4540.0,326.0,1.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,4,1,1,4
2013-01-05,1115.0,6.0,4771.0,339.0,1.0,0.0,1.0,5350.0,0,1,...,0,0,0,0,0,0,5,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-06-26,127.0,4.0,3335.0,480.0,1.0,0.0,0.0,1350.0,1,1,...,0,0,0,0,0,0,3,2,6,26
2014-06-27,127.0,5.0,4724.0,591.0,1.0,0.0,0.0,1350.0,1,1,...,0,0,0,0,0,0,4,2,6,27
2014-06-28,127.0,6.0,5398.0,716.0,1.0,0.0,0.0,1350.0,1,1,...,0,0,0,0,0,0,5,2,6,28


In [58]:
expanded.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpened',
       ...
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'dayofweek', 'quarter', 'month', 'dayofmonth'],
      dtype='object', length=721)

In [59]:
#create average sales per customer
df=expanded.groupby(by=['Store']).agg({'Sales':'sum','Customers':'sum'})
df['av_SalesPerCustomer']=df['Sales']/df['Customers']
df.sort_values(by='av_SalesPerCustomer',ascending=False)
expanded = pd.merge(expanded, df['av_SalesPerCustomer'], on=['Store'])

#create sales per customer for each store and day
#expanded['SalesPerCustomer']=expanded['Sales']/expanded['Customers']


In [60]:
#create av sales per day of the week
df=expanded.groupby(by=['Store','dayofweek']).agg({'Sales':'sum','Customers':'sum'})
df['av_SalesPerCustomer_dayofweek']=df['Sales']/df['Customers']
expanded = pd.merge(expanded, df['av_SalesPerCustomer_dayofweek'], on=['Store','dayofweek'])


In [61]:
#create av sales per day of month
df=expanded.groupby(by=['Store','dayofmonth']).agg({'Sales':'sum','Customers':'sum'})
df['av_SalesPerCustomer_dayofmonth']=df['Sales']/df['Customers']
expanded = pd.merge(expanded, df['av_SalesPerCustomer_dayofmonth'], on=['Store','dayofmonth'])


In [63]:
#drop if any other values null
data_new=expanded
data_new=data_new.dropna(axis = 0, how ='any')
data_new=data_new[data_new['Sales'] >0 ]
#expanded_new.to_csv('data/train_all_10_Oct.csv')